In [ ]:
import Utils
import pyDOE
import pandas as pd
import numpy as np
import scipy.stats.distributions as dist
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from collections import namedtuple
from sklearn.preprocessing import MinMaxScaler

## Helper Functions

In [2]:
''' KNN Regression Implementation for Minimax Robustness'''
def KNN_minimax(train_data,test_data,n):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    regr = KNeighborsRegressor(n_neighbors=n,weights='distance',algorithm='brute',p=2
                               ).fit(scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,11])

    pred = regr.predict(scaler.transform(test_data))
    return regr,pred

''' KNN Regression Implementation for Minimax Robustness'''
def KNN_composite(train_data,test_data,n):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    regr = KNeighborsRegressor(n_neighbors=n,weights='distance',algorithm='brute',p=2
                               ).fit(scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,-1])

    pred = regr.predict(scaler.transform(test_data))
    return regr,pred

## Load Training and Test Data Set initially Generated

In [3]:
X = [50,100,150,200,250,300,350,400,450,500]
Kriging_minimax = np.zeros([len(X),1],dtype=int)
Kriging_composite = np.zeros([len(X),1],dtype=int)
for k in range(len(X)):
    print (X[k])
    train = pd.read_csv(sys.path[1][:-3]+str('Training_Data_Sets\\'+str(X[k])+'Samples.csv')).iloc[:,1:]
    test = pd.read_csv(sys.path[1][:-3]+str('Test_Data_Sets\\750Samples.csv')).iloc[:,1:]
    true_minimax = np.array(test.True_Minimax)
    true_composite = np.array(test.True_Composite)
    Hyper_Parameters = np.arange(2,X[k])
    mean_abs_error_minimax = np.zeros(Hyper_Parameters.shape[0])
    mean_abs_error_composite = np.zeros(Hyper_Parameters.shape[0])
    for i in range(Hyper_Parameters.shape[0]):
        temp = Hyper_Parameters[i]
        model_minimax,pred_k_m = KNN_minimax(train,test.iloc[:,:10],np.ceil(temp).astype(int))
        model_composite,pred_k_c = KNN_composite(train,test.iloc[:,:10],np.ceil(temp).astype(int))
        mean_abs_error_minimax[i] = np.mean((abs(true_minimax-pred_k_m) / abs(true_minimax) ) * 100)
        mean_abs_error_composite[i] = np.mean((abs( true_composite-pred_k_c) / abs(true_composite) ) * 100)
    Kriging_minimax [k,:] = Hyper_Parameters[np.argmin(mean_abs_error_minimax)]
    Kriging_composite [k,:] = Hyper_Parameters[np.argmin(mean_abs_error_composite)]

50
100
150
200
250
300
350
400
450
500


In [4]:
Kriging_minimax

array([[ 5],
       [14],
       [ 9],
       [13],
       [ 8],
       [13],
       [16],
       [14],
       [ 9],
       [ 8]])

In [5]:
Kriging_composite

array([[ 46],
       [ 68],
       [146],
       [  8],
       [  9],
       [  6],
       [ 11],
       [  6],
       [  7],
       [  4]])